# ReverseNumber Example - two users

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, json

from colink import CoLink

from aiflows.utils import serve_utils
from aiflows.utils.general_helpers import read_yaml_file
from aiflows.messages import FlowMessage
from aiflows.utils import coflows_utils, colink_utils
from aiflows.workers import run_dispatch_worker_thread, run_get_instance_worker_thread

def status(cl):
    colink_utils.print_served_flows(cl, print_values=False)
    print("\nAll flow instances:")
    colink_utils.print_flow_instances(cl)

# Create two users

In [3]:
cls = colink_utils.start_colink_server_with_users(num_users=2)

# USER 0

In [4]:
run_dispatch_worker_thread(cls[0])
run_get_instance_worker_thread(cls[0])

Dispatch worker started in attached thread.
dispatch_point: coflows_dispatch
get_instances worker started in attached thread for user  02c5f78f4d27fa03c1c19b113a95ec847d17613b8ffb07d2876542df3120b793b2

~~~ serving get_instances request ~~~
Mounted 9a2c5b08-857f-4e43-b2c0-3b95cba41336 at flows:reverse_number_atomic:mounts:0300dd5667bb8eb8b25849121c3cb8eba6818d539a0294c59586921a4b612a9409:9a2c5b08-857f-4e43-b2c0-3b95cba41336
Fetched singleton 9a2c5b08-857f-4e43-b2c0-3b95cba41336

~~~ Dispatch task ~~~
flow_type: reverse_number_atomic
flow_id: 9a2c5b08-857f-4e43-b2c0-3b95cba41336
message_paths: ['push_tasks:c0e9825f-f99a-495b-a52d-5d2542a7b4be:msg']
parallel_dispatch: False

Called ReverseNumberAtomic 
 state {}

~~~ Dispatch task ~~~
flow_type: reverse_number_atomic
flow_id: 9a2c5b08-857f-4e43-b2c0-3b95cba41336
message_paths: ['push_tasks:d40025d4-9592-4884-9813-9a3166c0f34b:msg']
parallel_dispatch: False

Called ReverseNumberAtomic 
 state {}


In [5]:
serve_utils.serve_flow(
    cl=cls[0],
    flow_class_name="ReverseNumberFlowModule.ReverseNumberAtomicFlow",
    flow_endpoint="reverse_number_atomic",
    singleton=True,
)

Started serving ReverseNumberFlowModule.ReverseNumberAtomicFlow at flows:reverse_number_atomic.
dispatch_point: coflows_dispatch
parallel_dispatch: False
singleton: True


True

# USER 1

In [6]:
run_dispatch_worker_thread(cls[1])
run_get_instance_worker_thread(cls[1])

Dispatch worker started in attached thread.
dispatch_point: coflows_dispatch
get_instances worker started in attached thread for user  0300dd5667bb8eb8b25849121c3cb8eba6818d539a0294c59586921a4b612a9409

~~~ get_instances initiator ~~~
Received subflow instances: {
    "first_reverse_flow": {
        "flow_id": "9a2c5b08-857f-4e43-b2c0-3b95cba41336",
        "successful": true,
        "message": "Fetched local flow instance."
    },
    "second_reverse_flow": {
        "flow_id": "9a2c5b08-857f-4e43-b2c0-3b95cba41336",
        "successful": true,
        "message": "Fetched local flow instance."
    }
}

~~~ Dispatch task ~~~
flow_type: reverse_number_sequential
flow_id: 57c8335e-ae28-48dc-ad51-b4786aabd8ba
message_paths: ['push_tasks:7dc92d0e-184e-471e-8521-e66e81193d48:msg']
parallel_dispatch: False

Called ReverseNumberSequential 
 state {'current_call': 'first_reverse_flow'}

~~~ Dispatch task ~~~
flow_type: reverse_number_sequential
flow_id: 57c8335e-ae28-48dc-ad51-b4786aabd8ba
me

In [7]:
serve_utils.serve_flow(
    cl=cls[1],
    flow_class_name="ReverseNumberFlowModule.ReverseNumberSequentialFlow",
    flow_endpoint="reverse_number_sequential",
)

Started serving ReverseNumberFlowModule.ReverseNumberSequentialFlow at flows:reverse_number_sequential.
dispatch_point: coflows_dispatch
parallel_dispatch: False
singleton: False


True

In [8]:
# since we are creating new dummy users on every run, we inject user_id into config manually
# typically this would be hardcoded in yaml config
cfg_overrides = {
    "subflows_config": {
        "first_reverse_flow": {
            "user_id": cls[0].get_user_id()
        },
        "second_reverse_flow": {
            "user_id": cls[0].get_user_id()
        }
    }
}

flow = serve_utils.get_flow_instance(
    cl=cls[1],
    flow_endpoint="reverse_number_sequential",
    user_id="local",
    config_overrides = cfg_overrides,
)

Mounted 57c8335e-ae28-48dc-ad51-b4786aabd8ba at flows:reverse_number_sequential:mounts:local:57c8335e-ae28-48dc-ad51-b4786aabd8ba


In [9]:
input_data = {"id": 0, "number": 1234}
    
input_message = flow.package_input_message(input_data)
future = flow.get_reply_future(input_message)
reply_data = future.get_data()

print("Data sent:\n",  input_data, "\n")
print("REPLY:\n", reply_data, "\n")

Data sent:
 {'id': 0, 'number': 1234} 

REPLY:
 {'output_number': 1234} 



# Observe storage of both users

User 0 has a serve endpoint for ReverseNumberAtomic flow and a singleton instance of that flow.
User 1 has a serve endpoint for ReverseNumberSequential flow and an instance of that flow whose subflows are remote instances of the ReverseNumberAtomic at user 0.

In [10]:
status(cls[0])

 reverse_number_atomic/
   default_dispatch_point
   parallel_dispatch
   flow_class_name
   mounts/
     0300dd5667bb8eb8b25849121c3cb8eba6818d539a0294c59586921a4b612a9409/
       9a2c5b08-857f-4e43-b2c0-3b95cba41336/
         init
         config_overrides
         state
   init
   singleton

All flow instances:
 9a2c5b08-857f-4e43-b2c0-3b95cba41336


In [11]:
status(cls[1])

 reverse_number_sequential/
   flow_class_name
   init
   parallel_dispatch
   mounts/
     local/
       57c8335e-ae28-48dc-ad51-b4786aabd8ba/
         config_overrides
         init
         state
   singleton
   default_dispatch_point

All flow instances:
 57c8335e-ae28-48dc-ad51-b4786aabd8ba
